In [1]:
import polars as pl
from datetime import datetime

In [54]:
sprints = pl.read_csv("./data/sprints-Table 1.csv", separator=";")
entities = pl.read_csv("./data/data_for_spb_hakaton_entities1-Table 1.csv", separator=";")
history = pl.read_csv("./data/history-Table 1.csv", separator=";")
history.head()

entity_id,history_property_name,history_date,history_version,history_change_type,history_change,Столбец1,
i64,str,str,i64,str,str,str,str
94297,"""Время решения 3ЛП ФАКТ""","""9/10/24 11:17""",1,"""FIELD_CHANGED""","""<empty> -> 2024-09-10 11:17:06…",null,null
94297,"""Время решения (ФАКТ)""","""9/10/24 11:17""",1,"""FIELD_CHANGED""","""<empty> -> 2024-09-10 11:17:06…",null,null
94297,"""Исполнитель""","""7/13/23 11:07""",1,"""FIELD_CHANGED""","""user409017mail@mail.com -> use…",null,null
null,null,null,null,null,null,null,null
94297,"""Исполнитель""","""7/21/23 11:06""",3,"""FIELD_CHANGED""","""user4080458@mail.com -> user40…",null,null


In [3]:
def transform_date(x: str):
    date, time = x.split(" ")
    m, d, y = map(lambda x: x if len(x) == 2 else f'0{x}', date.split("/"))
    H, M = map(lambda x: x if len(x) == 2 else f'0{x}', time.split(":"))
    return str(datetime.strptime(f'{m}/{d}/{y} {H}:{M}', '%m/%d/%y %H:%M'))

In [4]:
ARROW = r' -> '
ARROW_LEN = len(ARROW)

def remove_empty(x: str):
    index = x.find(ARROW)
    if x.find(ARROW):
        return x[index + ARROW_LEN:]
    return x

In [50]:
entities

entity_id,area,type,status,state,priority,ticket_number,name,create_date,created_by,update_date,updated_by,parent_ticket_id,assignee,owner,due_date,rank,estimation,spent,workgroup,resolution
i64,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,i64,i64,str,str
94297,"""Система.Таск-трекер""","""Дефект""","""Закрыто""","""Normal""","""Средний""","""PPTS-1965""","""[FE] Бэклог. Кастомизация коло…","""2023-03-16 16:59:00.000000""","""А. К.""","""2024-09-10 11:20:09.193785""","""А. К.""",72779,"""А. К.""","""А. К.""",null,"""0|qzzywk:""",60,null,null,"""Готово"""
102481,"""Система.Ошибки""","""История""","""Закрыто""","""Normal""","""Критический""","""PPIN-1175""","""[ГенераторДокументов] Интеграц…","""2023-05-12 13:33:55.918127""","""А. З.""","""2024-08-06 19:30:16.692683""",null,3488105,"""А. Е.""","""А. Е.""",null,"""0|qv7n1c:y""",432000,null,"""Новая функциональность""","""Готово"""
1805925,"""Система.Таск-трекер""","""Дефект""","""Тестирование""","""Normal""","""Высокий""","""PPTS-3189""","""[FE] История изменений. Пустые…","""2023-07-12 09:36:04.479760""","""А. К.""","""2024-11-05 15:02:00.900484""","""А. К.""",null,"""А. К.""","""А. К.""",null,"""0|qzsklw:""",60,null,null,null
1934905,"""Система.Таск-трекер""","""Дефект""","""Закрыто""","""Normal""","""Средний""","""PPTS-3383""","""[FE] Зависимые поля. Тип реакц…","""2023-08-04 11:32:25.829919""","""А. Д.""","""2024-08-13 11:46:20.165757""","""А. К.""",null,"""А. К.""","""Я. П.""",null,"""0|qzh3e8:""",null,null,null,"""Готово"""
1943849,"""Система.Ошибки""","""Дефект""","""Отклонен исполнителем""","""Normal""","""Низкий""","""PPIN-1609""","""[BE] При сортировке по Теме ес…","""2023-08-09 06:20:44.391950""","""А. С.""","""2024-07-18 08:23:39.657571""","""А. Е.""",2083371,"""А. М.""","""А. Ж.""",null,"""0|qzzywj:zzzzzzzzzzzzzzzzzzzzz…",null,null,null,"""Отклонено"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5179477,"""Управление релизами изменениям…","""Задача""","""Закрыто""","""Normal""","""Средний""","""PPRC-2511""","""[PPPL] [FE] - Исследовать ui-k…","""2024-09-24 12:12:21.879096""","""Е. Б.""","""2024-09-27 06:13:08.773244""","""В. М.""",4933112,"""В. М.""","""Е. Б.""",null,"""0|qm0tow:""",57600,null,"""Новая функциональность""",null
5179714,"""Управление релизами изменениям…","""Задача""","""Закрыто""","""Normal""","""Средний""","""PPRC-2512""","""[PPPL] [FE] - Написать базовую…","""2024-09-24 12:18:13.307422""","""Е. Б.""","""2024-10-08 06:35:24.465539""","""В. М.""",4933112,"""В. М.""","""Е. Б.""",null,"""0|qm0sbk:""",86400,null,"""Новая функциональность""","""Готово"""
5179793,"""Управление релизами изменениям…","""Задача""","""Закрыто""","""Normal""","""Средний""","""PPRC-2513""","""[PPPL] [FE] - Сделать роутинг…","""2024-09-24 12:19:51.036183""","""Е. Б.""","""2024-10-28 06:43:55.730260""","""В. М.""",4933112,"""В. М.""","""Е. Б.""",null,"""0|qm0rw0:""",28800,null,"""Новая функциональность""","""Готово"""


{94297: 'PPTS-1965',
 102481: 'PPIN-1175',
 1805925: 'PPTS-3189',
 1934905: 'PPTS-3383',
 1943849: 'PPIN-1609',
 1966759: 'PPRC-1195',
 2353124: 'PPAR-3694',
 2383243: 'PPAR-3777',
 2441405: 'PPRC-1466',
 2444222: 'PPAR-3875',
 2503655: 'PPIN-2166',
 2536878: 'PPWI-2827',
 2540151: 'PPAR-4085',
 2582366: 'PPTS-4479',
 2623949: 'PPWI-2981',
 2824925: 'PPIN-2444',
 2840563: 'PPTS-4895',
 2867204: 'PPFW-1314',
 2887761: 'PPAR-4466',
 2938657: 'PPTS-5013',
 2970791: 'PPTS-5064',
 2980909: 'PPIN-2512',
 2995479: 'PPAR-4648',
 3027722: 'PPTS-5186',
 3051159: 'PPTS-5239',
 3057885: 'PPIN-2571',
 3127873: 'PPFW-1632',
 3130955: 'PPAR-5104',
 3185895: 'PPWI-3760',
 3206897: 'PPWI-3847',
 3207618: 'PPWI-3852',
 3207737: 'PPTS-5575',
 3315188: 'PPTS-5773',
 3332524: 'PPTS-5822',
 3335475: 'PPWI-4125',
 3335507: 'PPWI-4126',
 3335563: 'PPWI-4129',
 3335577: 'PPWI-4130',
 3335613: 'PPWI-4131',
 3385299: 'PPTS-5888',
 3421421: 'PPAR-5747',
 3424881: 'PPAR-5766',
 3481977: 'PPTS-6028',
 3482014: 'PPT

In [67]:
ticket2id = dict(zip(entities["ticket_number"].to_list(), entities["entity_id"].to_list()))
entity_id2sprint_id = {}
sprint2id= {}#dict(zip(sprints["

def map_one_day(x):
    x = list(x)
    ent_id = int(x[0])
    sprint_id = entity_id2sprint_id.get(ent_id, 0)
    x[10] = sprint_id
    if x[1] == 'Связанные Задачи':
        if x[5] == 'isBlockedBy':
            if x[7] == x[0]:
                block_id = ticket2id.get(x[6], None)
                if block_id is None:
                    return None
                else:
                    x[5]=block_id
                    return tuple(x)
            return None
        elif x[5] == "":
            return tuple(x)
        else:
            return None
    elif x[1] == "Спринт":
        sprint_id = sprint2id.get(x[5], 0)
        entity_id2sprint_id[ent_id] = sprint_id
        x[10] = sprint_id
    return tuple(x)

In [72]:
def get_lf(path):
    return pl.scan_csv(path, separator=";")
    

In [68]:
def preprocess_history(history: pl.DataFrame, entities: pl.DataFrame, sprints: pl.DataFrame):
    history = get_lf("./data/data_for_spb_hakaton_entities1-Table 1.csv").filter(
        pl.col("entity_id").is_not_null() and pl.col("history_property_name").is_not_null()).select(
        "entity_id", "history_property_name", "history_date", "history_change").collect()
    history = history.with_columns(
        pl.col("history_date").str.to_datetime('%m/%d/%y %H:%M')
    )
    history = history.with_columns(
        pl.col("history_change").str.split(r' -> ').arr.get(-1)
    )
    history.replace_column(history.get_column_index('history_change'),
                           history['history_change'].map_elements(remove_empty))
    history = history.sort(['history_date'], maintain_order=True)
    
    history = history.with_columns(history.get_column('history_change').slice(1).append(pl.Series("_", ["final"])).alias('history_change_tmp'))
    history = history.with_columns(history.get_column('entity_id').slice(1).append(pl.Series("_", [0])).alias('entity_id_tmp'))
    
    history = history.group_by().map_groups()
    return history


In [69]:
history.filter(pl.col('history_change') == '')

entity_id,history_property_name,history_date,history_version,history_change_type,history_change,Столбец1,
i64,str,str,i64,str,str,str,str


In [70]:
processed = preprocess_history(history=history, entities=entities, sprints=sprints)


/tmp/ipykernel_3431134/2782802616.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  history.replace_column(history.get_column_index('history_date'), history['history_date'].map_elements(transform_date))
/tmp/ipykernel_3431134/2782802616.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  history.replace_column(history.get_column_index('history_change'), history['history_change'].map_elements(remove_empty))


In [71]:
processed.filter(pl.col('column_1')=="Связанные Задачи")

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7
i64,str,str,i64,str,str,str,i64
3671384,"""Связанные Задачи""","""2024-07-25 12:53:00""",62,"""FIELD_CHANGED""","""4600573""","""PPRC-2375""",3671384
3681209,"""Связанные Задачи""","""2024-07-11 11:40:00""",22,"""FIELD_CHANGED""","""3681164""","""PPTS-6300""",3681209
3799650,"""Связанные Задачи""","""2024-04-16 08:39:00""",3,"""FIELD_CHANGED""","""3493071""","""PPWI-4320""",3799650
4065300,"""Связанные Задачи""","""2024-07-03 09:21:00""",20,"""FIELD_CHANGED""","""4452397""","""PPAR-7408""",4065300
4083383,"""Связанные Задачи""","""2024-06-21 13:46:00""",28,"""FIELD_CHANGED""","""4326408""","""PPAR-7162""",4083383
4206312,"""Связанные Задачи""","""2024-07-17 10:30:00""",21,"""FIELD_CHANGED""","""4546580""","""PPTS-7894""",4206312
4501324,"""Связанные Задачи""","""2024-07-11 11:32:00""",9,"""FIELD_CHANGED""","""4441131""","""PPWI-5348""",4501324
4533703,"""Связанные Задачи""","""2024-07-16 14:11:00""",8,"""FIELD_CHANGED""","""4417366""","""PPTS-7659""",4533703
4686828,"""Связанные Задачи""","""2024-08-05 11:21:00""",6,"""FIELD_CHANGED""","""3725698""","""PPWI-4517""",4686828


In [18]:
processed

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7
i64,str,str,i64,str,str,str,i64
94297,"""Исполнитель""","""2023-07-13 11:07:00""",1,"""FIELD_CHANGED""","""user408045mail@mail.com""","""user4091341@mail.com""",94297
94297,"""Исполнитель""","""2023-07-21 11:06:00""",3,"""FIELD_CHANGED""","""user4091341@mail.com""","""user4083841@mail.com""",94297
94297,"""Исполнитель""","""2024-09-10 11:16:00""",5,"""FIELD_CHANGED""","""user4083841@mail.com""","""Спринт 2024.3.5.NPP Shared Spr…",94297
94297,"""Спринт""","""2024-09-10 11:16:00""",6,"""FIELD_CHANGED""","""Спринт 2024.3.5.NPP Shared Spr…","""analysis""",94297
94297,"""Статус""","""2024-09-10 11:16:00""",7,"""FIELD_CHANGED""","""analysis""","""2024-09-10 11:17:06.680223""",94297
…,…,…,…,…,…,…,…
5179881,"""Спринт""","""2024-10-23 07:00:00""",11,"""FIELD_CHANGED""","""Спринт 2024.4.1.NPP Shared Spr…","""Спринт 2024.4.3.NPP Shared Spr…",5179881
5179881,"""Спринт""","""2024-10-23 07:00:00""",11,"""FIELD_CHANGED""","""Спринт 2024.4.3.NPP Shared Spr…","""Готово""",5179881
5179881,"""Резолюция""","""2024-10-23 07:00:00""",12,"""FIELD_CHANGED""","""Готово""","""closed""",5179881


In [141]:
processed.write_csv('./data_for_spb_hakaton_entities/test_sort.csv')